<a href="https://colab.research.google.com/github/VesleAnne/Barney_Stinson_Chatbot/blob/main/Prepairing_data_(langchain).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain huggingface_hub sentence-transformers pandas torch transformers chromadb -q #chromadb, nltk

In [ ]:
!pip install -U langchain-community -q
#!pip install chardet

In [ ]:
import pandas as pd
import torch
#import chromadb
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import AutoTokenizer, AutoModel
#import nltk
#import re
#import tqdm
#from nltk.tokenize import sent_tokenize
from langchain.vectorstores import Chroma
from langchain.schema import Document

In [ ]:
#import chardet

#def detect_encoding(file_path):
#    with open(file_path, 'rb') as file:
#        result = chardet.detect(file.read())
#        return result['encoding']

#encoding = detect_encoding(txt_path)
#print(f"Detected encoding: {encoding}")

In [ ]:
#nltk.download('punkt')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
%cd '/content/drive/MyDrive/МФТИ/nlp2/RAG'

/content/drive/MyDrive/МФТИ/nlp2/RAG


In [ ]:
current_directory = os.getcwd()
save_directory = os.path.join(current_directory, 'DATA')
os.makedirs(save_directory, exist_ok=True)

In [ ]:
# Initialize ChromaDB client
#client = chromadb.Client()

In [ ]:
# Create separate collections for text, Excel text, and Excel context
#text_collection = client.create_collection(name="text_embeddings")
#csv_text_collection = client.create_collection(name="csv_text_embeddings")

In [ ]:
# Initialize the multilingual model
model_name = "intfloat/multilingual-e5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

In [ ]:
# Initialize LangChain's Hugging Face embeddings with the multilingual model
hf = HuggingFaceEmbeddings(model_name=model_name)

In [ ]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

In [ ]:
txt_filename = 'Stinson Barney. The Bro Code - BooksCafe.Net.txt'
txt_path = os.path.join(current_directory, 'DATA', txt_filename)

In [ ]:
# Open the Bro Code
with open(txt_path, 'r', encoding='windows-1252') as file:
    text = file.read()

In [ ]:
# Define the headers to split on
headers_to_split_on = [
    ("INTRODUCTION", "Header 1"),
    ("BROCABULARY", "Header 2"),
    ("APPROPRIATE \"BRO\" USAGE", "Header 3"),
    ("INAPPROPRIATE \"BRO\" USAGE", "Header 4"),
    ("ORIGIN", "Header 5"),
    ("ARTICLE", "Header 6")
]

In [ ]:
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

md_header_splits = markdown_splitter.split_text(text)

In [ ]:
#Verifying that the file was splitted the reight way.
md_header_splits[15]

Document(metadata={'Header 6': '10'}, page_content='A Bro will drop whatever he\'s doing and rush to help his Bro dump a chick.  \nIt\'s normal for a Bro to get confused and disoriented when dumping a chick. For some reason he\'s worried she\'ll become agitated or even violent after he calmly explains his desire to have sex with her friends. This is when a Bro most needs his Bro to remind him that there are plenty of chicks in the ocean, and that a breakup need not be hazardous, stressful, or even time-consuming.  \nSIDE-BRO: HOW TO DUMP A CHICK IN SIX WORDS OR LESS  \n"Maybe try a side salad instead."  \n"Cute! You\'re growing a mustache, too!"  \n"She looks like a younger you."  \n"I will finance a boob job."  \n"Sorry I threw out your shoes."  \n"Your sister let me do that."')

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
csv_filename = 'Barney.csv'
csv_path = os.path.join(current_directory, 'DATA', csv_filename)

In [ ]:
df = pd.read_csv(csv_path)
df.head(10)

,name,text,context
0,Barney,"(on the phone) hey, so you know how I've alwa...",What was I doing? Your Uncle Marshall was tak...
1,Barney,"Okay, meet me at the bar in fifteen minutes, ...","Hey, you wanna do something tonight? Hey."
2,Barney,Where's your suit!? Just once when I say suit...,Hey. I did that one time.
3,Barney,It was a blazer!,"I did that one time. You know, ever since co..."
4,Barney,I see what this is about. Have you forgotten ...,"You know, ever since college it's been Marsha..."
5,Barney,"Ted, I'm going to teach you how to live. (Ted...",I see what this is about. Have you forgotten ...
6,Barney,"Lesson one, lose the goatee. It doesn't look ...","Oh, right. Hi. I'm not wearing a suit."
7,Barney,"Lesson two, get a suit. Suits are cool. (Poin...","I'm not wearing a suit. Thirty, right. You'r..."
8,Barney,"I thought I was your best friend. Ted, say I'...","Thirty, right. You're right. I guess it's jus..."
9,Barney,"Good! And as your best friend, I suggest we p...","You're my best friend, Barney. Wai-no, no, n..."


In [ ]:
csv_texts = df['text'].fillna("").astype(str).tolist()
len(csv_texts)

5040

In [ ]:
csv_texts[0]

" (on the phone) hey, so you know how I've always had a thing for half-Asian girls? Well, now I've got a new favorite: Lebanese girls! Lebanese girls are the new half-Asians."

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\'"],
    chunk_size=20,
    chunk_overlap=0.2,
    length_function=len
)

In [ ]:
csv_splits = [text_splitter.split_text(txt) for txt in csv_texts]
csv_splits[10]

[' (Taps a woman names Yasmine) Hi, have you met Ted? (Leaves and watches from a distance).']

In [ ]:
csv_data_documents = [Document(page_content=chunk) for chunks in csv_splits for chunk in chunks]

In [ ]:
question = "Hello, bro. Do you have a suit?"

In [ ]:
csv_filename = 'Barney.csv'
current_directory = os.getcwd()
csv_path = os.path.join(current_directory, 'DATA', csv_filename)
persist_directory_csv = os.path.join(current_directory, 'csv_data_db')

In [ ]:
persist_directory = current_directory
# Save .txt file chunks separately in one collection (Bro Code)
bro_code_splits = md_header_splits

In [ ]:
# Create vector database for the Bro Code (from .txt)
persist_directory_bro_code = os.path.join(persist_directory, 'bro_code_db')
vectordb_bro_code = Chroma.from_documents(
    documents=bro_code_splits,
    embedding=hf,
    persist_directory=persist_directory_bro_code
)

In [ ]:
# Create and persist vector database
persist_directory_csv = os.path.join(persist_directory, 'csv_data_db')
vectordb_csv = Chroma.from_documents(
    documents=csv_data_documents,
    embedding=hf,
    persist_directory=persist_directory_csv
)

In [ ]:
vectordb_csv.similarity_search(question,k=3)

[Document(metadata={}, page_content='Hi. Have you met me?'),
 Document(metadata={}, page_content='Hi. Have you met me?'),
 Document(metadata={}, page_content=' Hey, Ted. Whatcha doing?')]

In [ ]:
vectordb_bro_code.similarity_search(question,k=3)

[Document(metadata={'Header 6': '149'}, page_content='A Bro pretends to understand and enjoy cigars.  \nCIGAR WORD SEARCH'),
 Document(metadata={'Header 6': '49'}, page_content='Wften asked, "Do you need some help?" a Bro shall automatically respond, "I got it," whether or not he\'s actually got it.  \nEXCEPTIONS: Carrying an expensive TV, parallel parking an expensive car, loading an expensive TV into an expensive car.'),
 Document(metadata={'Header 6': '49'}, page_content='Wften asked, "Do you need some help?" a Bro shall automatically respond, "I got it," whether or not he\'s actually got it.  \nEXCEPTIONS: Carrying an expensive TV, parallel parking an expensive car, loading an expensive TV into an expensive car.')]

In [ ]:
vectordb_bro_code.max_marginal_relevance_search(question,k=2, fetch_k=3)

[Document(metadata={'Header 6': '149'}, page_content='A Bro pretends to understand and enjoy cigars.  \nCIGAR WORD SEARCH'),
 Document(metadata={'Header 6': '49'}, page_content='Wften asked, "Do you need some help?" a Bro shall automatically respond, "I got it," whether or not he\'s actually got it.  \nEXCEPTIONS: Carrying an expensive TV, parallel parking an expensive car, loading an expensive TV into an expensive car.')]

In [ ]:
vectordb_csv.max_marginal_relevance_search(question,k=2, fetch_k=3)